## 参考文献

https://users.aalto.fi/~vanvlm1/mne-rsa/auto_examples/plot_sensor_level.html


http://www.pymvpa.org/examples.html#chap-examples
http://www.pymvpa.org/tutorial_eventrelated_searchlight.html#chap-tutorial-eventrelated-searchlight
http://www.pymvpa.org/generated/mvpa2.measures.searchlight.Searchlight.html#mvpa2.measures.searchlight.Searchlight

several toolboxes have been developed that
implement the methods described in the rest of this paper; the PyMVPA toolbox (Hanke, Halchenko, Sederberg, Hanson, et al., 2009; www.pymvpa.org) handles both fMRI and M/EEG data using the open-source Python language (Hanke, Halchenko, Sederberg, Olivetti, et al., 2009)



https://mne.tools/stable/auto_examples/decoding/linear_model_patterns.html#ex-linear-patterns


In [ ]:
import mne
from mne import io, EvokedArray
from mne.datasets import sample
from mne.decoding import Vectorizer, get_coef

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# import a linear classifier from mne.decoding
from mne.decoding import LinearModel
import numpy as np

In [ ]:
list_epochs_all = list()

sub_ids = [

    "pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y",
    "pre4014_y","pre4015_y","pre4017_y",
    "pre4021_y","pre4022_y","pre4023_y","pre4026_y","pre4027_y","pre4028_y","pre4029_y","pre4030_y",


]
sub_ids = [

    "pre4008_y"

]

file_path = 'E:\实验\阅读困难\数据处理\pre4\data\\6epoch_clean_RSA\\'

for sub_id in sub_ids:
    data_path = file_path + sub_id + '_RSA-epo.fif'
    epochs_all = mne.read_epochs(fname=data_path)
    #做decoding要考虑被试内每个条件试次一样，如果被试间不一样可以用循环来解决。
    epochs_all.equalize_event_counts(method='mintime')
    list_epochs_all.append(epochs_all)

# 如果只想看某一部分的脑区,配合更改ndarray的维度
# channels_nieye = ['M1','T7','P7','TP7','M2','T8','P8','TP8','CP5','CP6','C5','C6']
# epochs_nieye = epochs_all.pick(picks=channels_nieye)

'''
思路，先求ERP，然后再给ndarray，然后再计算RDM
'''

In [ ]:
epochs_sub0 = list_epochs_all[0]
labels = epochs_sub0.events[:, -1]

data_sub0 =  epochs_sub0.get_data().reshape(len(labels), -1)

clf = LogisticRegression(solver="liblinear")  # liblinear is faster than lbfgs
scaler = StandardScaler()

# create a linear model with LogisticRegression
model = LinearModel(clf)

# fit the classifier on MEG data
X = scaler.fit_transform(data_sub0)
model.fit(X, labels)

In [ ]:
# Extract and plot spatial filters and spatial patterns
for name, coef in (("patterns", model.patterns_), ("filters", model.filters_)):
    # We fitted the linear model onto Z-scored data. To make the filters
    # interpretable, we must reverse this normalization step
    coef = scaler.inverse_transform([coef])[0]

    # The data was vectorized to fit a single model across all time points and
    # all channels. We thus reshape it:
    coef = coef.reshape(len(epochs.ch_names), -1)

    # Plot
    evoked = EvokedArray(coef, epochs.info, tmin=epochs.tmin)
    fig = evoked.plot_topomap()
    fig.suptitle(f"MEG {name}")